# Toy EDA

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
TRAIN_PATH = 'data/full/train'
IGNORE = ['.DS_Store']

train_images = []
train_labels = []

for label in os.listdir(TRAIN_PATH):
    if label in IGNORE:
        continue
    full_path = f'{TRAIN_PATH}/{label}'

    for filename in os.listdir(full_path):
        img = cv2.imread(os.path.join(full_path, filename))
        train_images.append(img)
        train_labels.append(label)

In [ ]:
train_images[0].shape

To display images, I use `np.flip(train_images[0], axis=-1)` to change coloring 

In [ ]:
plt.imshow(np.flip(train_images[100], axis=-1))
plt.title(train_labels[100]);

See how everything works with grey scale, so we could reduce number of features

In [ ]:
img = plt.imshow(train_images[100])
gray = lambda rgb : np.dot(rgb[... , :3] , [0.299 , 0.587, 0.114])
gray = gray(train_images[100])
plt.imshow(gray, cmap = plt.get_cmap(name = 'gray'))

In [ ]:
samples = len(train_images)
train_images_combined = np.array(train_images).reshape(samples, 224 * 224 * 3)

# Deep Learning Approach

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='data/full/train/', transform=transform)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=64*54*54, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=100)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN().cuda()

In [ ]:
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch {epoch + 1}, batch {i + 1}, loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished training')

In [ ]:
torch.save(model.state_dict(), "weights")

In [ ]:
test_dataset = datasets.ImageFolder(root='data/full/test/', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()

num_correct = 0
num_total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        num_total += labels.size(0)
        num_correct += (predicted == labels).sum().item()

# Compute the accuracy
accuracy = num_correct / num_total
print(f'Test accuracy: {accuracy:.3f}')

Bad accuracy, because:
- Not enough observations
- No control of weight initilization procedure
- Too many classes

## Pretrained CNN: ConvNet as fixed feature extractor

### Main class

In [ ]:
import time
import os

from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

class CNNClass():
    def __init__(self, model, transform, params, name,
                 path = 'data/full',
                 criterion = nn.CrossEntropyLoss(), 
                 optimizer = None,
                 device = 'cuda:0',
                 lr = 0.001):
        
        self.model = model
        self.transform = transform
        
        train_data = ImageFolder(f'{path}/train', transform=transform)
        valid_data = ImageFolder(f'{path}/valid', transform=transform)
        
        self.train_loader = DataLoader(train_data, batch_size=params['BATCH_SIZE'], shuffle=True, num_workers=2)
        self.valid_loader = DataLoader(valid_data, batch_size=params['BATCH_SIZE'], shuffle=True, num_workers=2)
        self.params = params
        self.name = name
        
        if not os.path.exists(f'weights/{self.name}'):
            os.mkdir(f'weights/{self.name}')

        self.criterion = criterion
        if optimizer == None:
            self.optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        else:
            self.optimizer = optimizer
        self.device = device
    
    def train(self):
        start_time = time.time()

        self.losses = {
            'train_loss': list(),
            'val_loss': list()
               }

        best_val_loss = float('inf')
        epoch_no_improvement = 0
        best_epoch = 1
        for epoch in range(self.params['EPOCHS']):
            running_loss = 0.0
            val_loss = 0.0
            self.model.train()
            for images, labels in self.train_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(images)
                loss = self.criterion(output, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item() * images.size(0)

            # Completed epoch, calculate validation error
            self.model.eval()
            with torch.no_grad():
                for images, labels in self.valid_loader:
                    images, labels = images.to(self.device), labels.to(self.device)
                    outputs = self.model(images)
                    loss = self.criterion(outputs, labels)
                    val_loss += loss.item() * images.size(0)

            epoch_loss = running_loss / len(self.train_loader.dataset)
            val_loss = val_loss / len(self.valid_loader.dataset)

            print(f'Epoch {epoch+1} Train loss: {epoch_loss:.3f}. Valid loss: {val_loss:.3f}')

            self.losses['train_loss'].append(epoch_loss)
            self.losses['val_loss'].append(val_loss)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch + 1
                epoch_no_improvement = 0
                state = {'model': self.model.state_dict(), 
                         'optimizer': self.optimizer.state_dict()}
                torch.save(state, f'weights/{self.name}/{self.name}.pth')
            else:
                epoch_no_improvement += 1
                
            if epoch_no_improvement == self.params['EARLY_STOP']:
                print(f'Training completed! No improvement last {epoch_no_improvement} epoches.' +
                      f'\nBest valid accuracy: {best_val_loss:.2f}' +
                      f'\nBest epoch: {best_epoch}')
                self.train_time = "%s" % (time.time() - start_time)
                break
        
        self.train_time = "%s" % (time.time() - start_time)
        self.save_info()
        print('-'*10)
        print(f'Test Accuracy: {self.test():.2f}%')   
                
    def test(self, path='data/full/test', debug=False):
        test_data = ImageFolder(path, transform=self.transform)
        test_loader = DataLoader(test_data, batch_size=self.params['BATCH_SIZE'], shuffle=True, num_workers=2)

        self.model.load_state_dict(torch.load(f'weights/{self.name}/{self.name}.pth', 
                                              map_location=self.device)['model'])
        self.optimizer.load_state_dict(torch.load(f'weights/{self.name}/{self.name}.pth',
                                                  map_location=self.device)['optimizer'])

        correct = 0
        total = 0
        self.model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy
    
    def save_info(self):
        # save time
        with open(f'weights/{self.name}/train_time.txt', 'w') as f:
            f.write(self.train_time)
        
        # save all stats
        with open(f'summary.csv', 'a') as f:
            f.write(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")},{self.name},{self.train_time},{self.test()}\n')
        
        # save logs
        epoches = [i + 1 for i in range(len(self.losses['train_loss']))]
        with open(f'weights/{self.name}/accuracy_log.csv', 'w') as f:
            f.write("epoch,train_loss,valid_loss\n")
            for epoch, train_loss, valid_loss in zip(epoches, 
                                                     self.losses['train_loss'], 
                                                     self.losses['val_loss']):
                f.write(f"{epoch},{train_loss:.3f},{valid_loss:.3f}\n")
                
    def get_training_time(self):
        with open(f'weights/{self.name}/train_time.txt','r') as f:
            for line in f:
                print(np.round(float(line) / 60))
    
    def plot_accuracy(self):
        if os.path.exists(f'weights/{self.name}/accuracy_log.csv'):
            data = pd.read_csv(f'weights/{self.name}/accuracy_log.csv')
            fig, axs = plt.subplots(1, 1)
            axs.grid(alpha=0.4)
            axs.plot(data['epoch'],
                     data['train_loss'], label ='Train Loss', color='black')
            axs.plot(data['epoch'],
                     data['valid_loss'], label='Valid Loss', color='green')
            axs.legend()
            axs.set_xlabel('Epoch')
            axs.set_ylabel('Loss')
            axs.set_title(f'Loss for {self.name} model')
            return axs
        else:
            raise Exception(f'Missing weights/{self.name}/accuracy_log.csv')

#### AlexNet (native)

In [ ]:
from torchvision.models import alexnet, AlexNet_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = AlexNet_Weights.DEFAULT
model_sample = alexnet(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 50,
                'EARLY_STOP': 5}

model1 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  name='alexnet_native')

In [ ]:
model1.train()

#### ResNet50 (native)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features, 100)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 50,
                'EARLY_STOP': 5}

model2 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='resnet50_native')

In [ ]:
model2.train()

#### ResNet152 (native)

In [ ]:
from torchvision.models import resnet152, ResNet152_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet152_Weights.DEFAULT
model_sample = resnet152(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features,
                            100)
model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model3 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  name='resnet152_native')

In [ ]:
model3.train()

#### Inception V3 (native)

In [ ]:
from torchvision.models import inception_v3, Inception_V3_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = Inception_V3_Weights.DEFAULT
model_sample = inception_v3(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features,
                      100)
model_sample.aux_logits=False
model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 50,
          'EARLY_STOP': 5}

model4 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  name='inception-v3_native')

In [ ]:
model4.train()

#### EfficientNet-B3 (native)

In [ ]:
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_B3_Weights.DEFAULT
model_sample = efficientnet_b3(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1536, out_features=100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 30,
          'EARLY_STOP': 4}

model5 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='efficientnet-b3_native')

In [ ]:
model5.train()

#### EfficientNet-B6 (native)

In [ ]:
from torchvision.models import efficientnet_b6, EfficientNet_B6_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_B6_Weights.DEFAULT
model_sample = efficientnet_b6(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=2304, out_features=100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 4}

model6 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='efficientnet-b6_native')

In [ ]:
model6.train()

#### EfficientNetV2-S (native)

In [ ]:
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_V2_S_Weights.DEFAULT
model_sample = efficientnet_v2_s(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model7 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  name='efficientnetV2-S_native')

In [ ]:
model7.train()

#### EfficientNetV2-L (native)

Here, I need to tell what the difference between previous models and this one

In [ ]:
from torchvision.models import efficientnet_v2_l, EfficientNet_V2_L_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_V2_L_Weights.DEFAULT
model_sample = efficientnet_v2_l(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model8 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  name='efficientnetV2-L_native')

In [ ]:
model8.train()

#### ViT-b-14 (native)

In [ ]:
from torchvision.models import vit_b_16, ViT_B_16_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ViT_B_16_Weights.DEFAULT
model_sample = vit_b_16(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.heads.head = nn.Linear(model_sample.heads.head.in_features, 100)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 5}

model9 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.001, momentum=0.9, weight_decay=0.03),
                  name='vit-b-16_native')

In [ ]:
model9.train()

#### ViT-l-14 (native)

In [ ]:
from torchvision.models import vit_l_16, ViT_L_16_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ViT_L_16_Weights.DEFAULT
model_sample = vit_l_16(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.heads.head = nn.Linear(model_sample.heads.head.in_features, 100)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model10 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.001, momentum=0.9, weight_decay=0.03),
                  name='vit-l-16_native')

In [ ]:
model10.train()

#### SuffleNet (x1) (native)

In [ ]:
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ShuffleNet_V2_X1_0_Weights.DEFAULT
model_sample = shufflenet_v2_x1_0(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features, 100)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 40,
          'EARLY_STOP': 5}

model11 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.0001, momentum=0.9),
                  name='shufflenet-v2-x1-0_native')

In [ ]:
model11.train()

**Conclusion**: good idea to try small models, not large

## Use ResNet to Replace Classifier with Custom NN

### ResNet: CHANGE 1 (Adam-0.0005-DLReDL-ReLU)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024, 100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model7 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='resnet50_changed_(Adam-0.0005-DLReDL)')

In [ ]:
model7.train()

### ResNet: CHANGE 2 (Adam-0.0005-DLReDLReDL-ReLU)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(512, 100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model7 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='resnet50_changed_(Adam-0.0005-DLReDLReDL)')

In [ ]:
model7.train()

### ResNet: CHANGE 3 (SGD-0.001-DLReDL-ReLU)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(1024, 100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model7 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.001, momentum=0.9),
                  name='resnet50_changed_classifier_(SGD-0.001-DLReDL)')

In [ ]:
model7.train()

### ResNet: CHANGE 4 (Adam-0.0005-DLReDL-ELU)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(2048, 1024),
    nn.ELU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024, 100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model7 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='resnet50_changed_(Adam-0.0005-DLReDL-ELU)')

In [ ]:
model7.train()

### ResNet: CHANGE 5 (Adam-0.0005-DLReDLReDL-ReLU)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(2048, 1500),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(1500, 750),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(750, 100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model7 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='resnet50_changed_classifier_(DLReDLReDL)_optimized')
model7.train()

### TEST MODEL

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(1024, 100)
)

model_sample.to(device_sample)

transform_sample = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Increased number of epochs

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model7 = CNNClass(model=model_sample,
                  transform=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.005, momentum=0.9),
                  name='resnet50_changed_classifier_(SGD-0.001-DLReDL)')
model7.train()

## Image augmentation on training dataset

### Main class

In [ ]:
import time
import os

from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

class CNNClassExtended(CNNClass):
    def __init__(self, model, transforms_, params, name,
                 path = 'data/full',
                 criterion = nn.CrossEntropyLoss(), 
                 optimizer = None,
                 device = 'cuda:0',
                 lr = 0.001):
        
        self.model = model
        
        train_data = ImageFolder(f'{path}/train', transform=transforms_[0])
        valid_data = ImageFolder(f'{path}/valid', transform=transforms_[1])
        
        self.train_loader = DataLoader(train_data, batch_size=params['BATCH_SIZE'], shuffle=True)
        self.valid_loader = DataLoader(valid_data, batch_size=params['BATCH_SIZE'], shuffle=True)
        self.params = params
        self.name = name
        
        if not os.path.exists(f'weights/{self.name}'):
            os.mkdir(f'weights/{self.name}')

        self.criterion = criterion
        if optimizer == None:
            self.optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        else:
            self.optimizer = optimizer
        self.device = device
    
    def test(self, path='data/full/test', transform=None, debug=False):
        
        if transform == None:
            transform = transform_valid = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
            
        test_data = ImageFolder(path, transform=transform)
        test_loader = DataLoader(test_data, batch_size=self.params['BATCH_SIZE'], shuffle=True, num_workers=2)

        self.model.load_state_dict(torch.load(f'weights/{self.name}/{self.name}.pth', 
                                              map_location=self.device)['model'])
        self.optimizer.load_state_dict(torch.load(f'weights/{self.name}/{self.name}.pth',
                                                  map_location=self.device)['optimizer'])

        correct = 0
        total = 0
        self.model.eval()
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy

#### Hard Augmentation

##### AlexNet (hard augmented)

In [ ]:
from torchvision.models import alexnet, AlexNet_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = AlexNet_Weights.DEFAULT
model_sample = alexnet(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model1 = CNNClassExtended(model=model_sample,
                  transforms_= transform_sample,
                  params=model_params,
                  name='alexnet_hard_aug')

In [ ]:
model1.train()

##### ResNet50 (hard augmented)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model2 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='resnet50_hard_aug')

In [ ]:
model2.train()

##### ResNet152 (hard augmented)

In [ ]:
from torchvision.models import resnet152, ResNet152_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet152_Weights.DEFAULT
model_sample = resnet152(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features,
                            100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model3 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='resnet152_hard_aug')

In [ ]:
model3.train()

###### Inception V3 (hard augmented)

In [ ]:
from torchvision.models import inception_v3, Inception_V3_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = Inception_V3_Weights.DEFAULT
model_sample = inception_v3(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features,
                      100)
model_sample.aux_logits=False
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model4 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='inception-v3_hard_aug')

In [ ]:
model4.train()

##### EfficientNet-B3 (hard augmented)

In [ ]:
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_B3_Weights.DEFAULT
model_sample = efficientnet_b3(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1536, out_features=100)
)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model5 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='efficientnet-b3_hard_aug')

In [ ]:
model5.train()

##### EfficientNet-B6 (hard augmented)

In [ ]:
from torchvision.models import efficientnet_b6, EfficientNet_B6_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_B6_Weights.DEFAULT
model_sample = efficientnet_b6(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=2304, out_features=100)
)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 4}

model6 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='efficientnet-b6_hard_aug')

In [ ]:
model6.train()

##### EfficientNetV2-S (hard augmented)

In [ ]:
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_V2_S_Weights.DEFAULT
model_sample = efficientnet_v2_s(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model7 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='efficientnetV2-S_hard_aug')

In [ ]:
model7.train()

##### EfficientNetV2-L (hard augmented)

Here, I need to tell what the difference between previous models and this one

In [ ]:
from torchvision.models import efficientnet_v2_l, EfficientNet_V2_L_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_V2_L_Weights.DEFAULT
model_sample = efficientnet_v2_l(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model8 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='efficientnetV2-L_hard_aug')

In [ ]:
model8.train()

##### ViT-b-14 (hard augmented)

In [ ]:
from torchvision.models import vit_b_16, ViT_B_16_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ViT_B_16_Weights.DEFAULT
model_sample = vit_b_16(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.heads.head = nn.Linear(model_sample.heads.head.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 5}

model9 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.001, momentum=0.9, weight_decay=0.03),
                  name='vit-b-16_hard_aug')

In [ ]:
model9.train()

##### ViT-l-14 (hard augmented)

In [ ]:
from torchvision.models import vit_l_16, ViT_L_16_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ViT_L_16_Weights.DEFAULT
model_sample = vit_l_16(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.heads.head = nn.Linear(model_sample.heads.head.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model10 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.001, momentum=0.9, weight_decay=0.03),
                  name='vit-l-16_hard_aug')

In [ ]:
model10.train()

##### SuffleNet (x1) (hard augmented)

In [ ]:
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ShuffleNet_V2_X1_0_Weights.DEFAULT
model_sample = shufflenet_v2_x1_0(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomEqualize(p=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomPerspective(p=0.3, distortion_scale=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 5}

model11 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.0001, momentum=0.9),
                  name='shufflenet-v2-x1-0_hard_aug')

In [ ]:
model11.train()

#### Sorf Augmentation

##### AlexNet (soft augmented)

In [ ]:
from torchvision.models import alexnet, AlexNet_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = AlexNet_Weights.DEFAULT
model_sample = alexnet(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model1 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='alexnet_soft_aug')

In [ ]:
model1.train()

##### ResNet50 (soft augmented)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet50_Weights.DEFAULT
model_sample = resnet50(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model2 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='resnet50_soft_aug')

In [ ]:
model2.train()

##### ResNet152 (soft augmented)

In [ ]:
from torchvision.models import resnet152, ResNet152_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet152_Weights.DEFAULT
model_sample = resnet152(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features,
                            100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model3 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='resnet152_soft_aug')

In [ ]:
model3.train()

##### Inception V3 (soft augmented)

In [ ]:
from torchvision.models import inception_v3, Inception_V3_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = Inception_V3_Weights.DEFAULT
model_sample = inception_v3(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features,
                      100)
model_sample.aux_logits=False
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model4 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='inception-v3_soft_aug')

In [ ]:
model4.train()

##### EfficientNet-B3 (soft augmented)

In [ ]:
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_B3_Weights.DEFAULT
model_sample = efficientnet_b3(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1536, out_features=100)
)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 4}

model5 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='efficientnet-b3_soft_aug')

In [ ]:
model5.train()

##### EfficientNet-B6 (soft augmented)

In [ ]:
from torchvision.models import efficientnet_b6, EfficientNet_B6_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_B6_Weights.DEFAULT
model_sample = efficientnet_b6(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=2304, out_features=100)
)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 4}

model6 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.Adam(model_sample.parameters(), lr=0.0005),
                  name='efficientnet-b6_soft_aug')

In [ ]:
model6.train()

##### EfficientNetV2-S (soft augmented)

In [ ]:
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_V2_S_Weights.DEFAULT
model_sample = efficientnet_v2_s(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model7 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='efficientnetV2-S_soft_aug')

In [ ]:
model7.train()

##### EfficientNetV2-L (soft augmented)

In [ ]:
from torchvision.models import efficientnet_v2_l, EfficientNet_V2_L_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = EfficientNet_V2_L_Weights.DEFAULT
model_sample = efficientnet_v2_l(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.classifier[-1] = nn.Linear(model_sample.classifier[-1].in_features,
                                        100)
model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model8 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  name='efficientnetV2-L_soft_aug')

In [ ]:
model8.train()

##### ViT-b-14 (soft augmented)

In [ ]:
from torchvision.models import vit_b_16, ViT_B_16_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ViT_B_16_Weights.DEFAULT
model_sample = vit_b_16(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.heads.head = nn.Linear(model_sample.heads.head.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 5}

model9 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.001, momentum=0.9, weight_decay=0.03),
                  name='vit-b-16_soft_aug')

In [ ]:
model9.train()

##### ViT-l-14 (soft augmented)

In [ ]:
from torchvision.models import vit_l_16, ViT_L_16_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ViT_L_16_Weights.DEFAULT
model_sample = vit_l_16(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.heads.head = nn.Linear(model_sample.heads.head.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
                'EPOCHS': 100,
                'EARLY_STOP': 5}

model10 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.001, momentum=0.9, weight_decay=0.03),
                  name='vit-l-16_soft_aug')

In [ ]:
model10.train()

##### SuffleNet (x1) (soft augmented)

In [ ]:
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

device_sample = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ShuffleNet_V2_X1_0_Weights.DEFAULT
model_sample = shufflenet_v2_x1_0(weights=weights)

for param in model_sample.parameters():
    param.requires_grad = False

model_sample.fc = nn.Linear(model_sample.fc.in_features, 100)

model_sample.to(device_sample)

transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_sample = [transform_test, transform_valid]

model_params = {'BATCH_SIZE': 32,
          'EPOCHS': 100,
          'EARLY_STOP': 5}

model11 = CNNClassExtended(model=model_sample,
                  transforms_=transform_sample,
                  params=model_params,
                  optimizer = optim.SGD(model_sample.parameters(), lr=0.0001, momentum=0.9),
                  name='shufflenet-v2-x1-0_soft_aug')

In [ ]:
model11.train()

# Some Code to Fetch Labels

In [ ]:
from PIL import Image
import numpy as np

test_image = Image.open('data/full/valid/bmx/4.jpg')

model.eval()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transformed_image = transform(test_image)
batch_image = torch.unsqueeze(transformed_image, 0)
output = model(batch_image)

idx = np.argmax(torch.nn.functional.softmax(output, dim=1)[0].detach().numpy())
train_loader.dataset.classes[idx]

model.eval()
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor()
])
transformed_image = transform(test_image)
batch_image = torch.unsqueeze(transformed_image, 0)
output = model(batch_image)

idx = np.argmax(torch.nn.functional.softmax(output, dim=1)[0].detach().numpy())
train_loader.dataset.classes[idx]